In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.preprocessing import MultiLabelBinarizer

# Predefined specializations
specializations = [
    "Criminal Law", "Business Law", "Family Law", "Labor Law", 
    "Civil Rights Law", "Tax Law", "Real Estate Law", "Intellectual Property Law", 
    "Bankruptcy Law", "Personal Injury Law", "Environmental Law", "Estate Planning Law", 
    "Corporate Law", "Immigration Law", "Contract Law"
]

def generate_lawyers(n=50, max_specs=5):
    lawyers = []
    for i in range(n):
        num_specs = np.random.randint(1, max_specs+1)
        lawyers.append({
            "lawyer_id": f"lawyer_{i+1}",
            "specializations": list(np.random.choice(specializations, num_specs, replace=False))
        })
    return pd.DataFrame(lawyers)

def generate_clients(n=20, max_prefs=5):
    clients = []
    for i in range(n):
        num_prefs = np.random.randint(1, max_prefs+1)
        clients.append({
            "client_id": f"client_{i+1}",
            "preferences": list(np.random.choice(specializations, num_prefs, replace=False))
        })
    return pd.DataFrame(clients)

lawyers_df = generate_lawyers()
clients_df = generate_clients()

# Binarize the specializations and preferences
mlb = MultiLabelBinarizer()
lawyers_specializations = mlb.fit_transform(lawyers_df['specializations'])
clients_preferences = mlb.transform(clients_df['preferences'])

# Convert to DataFrames
lawyers_features = pd.DataFrame(lawyers_specializations, columns=mlb.classes_)
clients_features = pd.DataFrame(clients_preferences, columns=mlb.classes_)


In [6]:
embedding_dimension = 32

# Model to compute lawyer and client embeddings
class LawyerRecommenderModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.lawyer_model = tf.keras.Sequential([
            tf.keras.layers.Dense(embedding_dimension, activation='relu'),
            tf.keras.layers.Dense(embedding_dimension, activation='relu')
        ])
        self.client_model = tf.keras.Sequential([
            tf.keras.layers.Dense(embedding_dimension, activation='relu'),
            tf.keras.layers.Dense(embedding_dimension, activation='relu')
        ])
        self.task = tfrs.tasks.Retrieval()

    def compute_loss(self, features, training=False):
        lawyer_embeddings = self.lawyer_model(features['lawyer_features'])
        client_embeddings = self.client_model(features['client_features'])
        return self.task(client_embeddings, lawyer_embeddings)

# Corrected dataset combination
def combine_features(lawyer_features, client_features):
    return {"lawyer_features": lawyer_features["lawyer_features"], "client_features": client_features["client_features"]}

combined_dataset = tf.data.Dataset.zip((tf_lawyers, tf_clients)).map(combine_features)

# Train the model
model = LawyerRecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model.fit(combined_dataset, epochs=10)



Epoch 1/10


2/2 [==============================] - 0s 3ms/step - loss: 22.5182 - regularization_loss: 0.0000e+00 - total_loss: 22.5182
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 19.3093 - regularization_loss: 0.0000e+00 - total_loss: 19.3093
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 17.8438 - regularization_loss: 0.0000e+00 - total_loss: 17.8438
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 17.7848 - regularization_loss: 0.0000e+00 - total_loss: 17.7848
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 13.9512 - regularization_loss: 0.0000e+00 - total_loss: 13.9512
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 14.4211 - regularization_loss: 0.0000e+00 - total_loss: 14.4211
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 11.5511 - regularization_loss: 0.0000e+00 - total_loss: 11.5511
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 8.0

In [8]:
def recommend_lawyers(client_id, num_recommendations=5):
    client_idx = int(client_id.split('_')[1]) - 1
    client_features = clients_features.iloc[client_idx:client_idx+1]
    client_embedding = model.client_model.predict(client_features)

    # Compute similarities
    lawyer_embeddings = model.lawyer_model.predict(lawyers_features)
    similarities = tf.linalg.matmul(client_embedding, lawyer_embeddings, transpose_b=True)
    top_indices = tf.argsort(similarities, axis=-1, direction='DESCENDING')[0, :num_recommendations].numpy()

    return lawyers_df.iloc[top_indices]

# Test the recommendation function
test_client_id = 'client_1'
recommended_lawyers = recommend_lawyers(test_client_id)
print(f"Recommended Lawyers for {test_client_id}:\n{recommended_lawyers}")
recommended_lawyers


2/2 [==============================] - 0s 2ms/step
Recommended Lawyers for client_1:
    lawyer_id                                    specializations
0    lawyer_1        [Criminal Law, Family Law, Real Estate Law]
12  lawyer_13                                  [Immigration Law]
11  lawyer_12                    [Real Estate Law, Criminal Law]
32  lawyer_33  [Family Law, Business Law, Immigration Law, Ta...
41  lawyer_42  [Business Law, Family Law, Civil Rights Law, I...


,lawyer_id,specializations
0,lawyer_1,"[Criminal Law, Family Law, Real Estate Law]"
12,lawyer_13,[Immigration Law]
11,lawyer_12,"[Real Estate Law, Criminal Law]"
32,lawyer_33,"[Family Law, Business Law, Immigration Law, Ta..."
41,lawyer_42,"[Business Law, Family Law, Civil Rights Law, I..."


In [9]:
clients_df

,client_id,preferences
0,client_1,"[Family Law, Criminal Law]"
1,client_2,"[Environmental Law, Contract Law, Family Law, ..."
2,client_3,"[Environmental Law, Family Law, Civil Rights Law]"
3,client_4,"[Environmental Law, Labor Law]"
4,client_5,"[Civil Rights Law, Bankruptcy Law]"
5,client_6,[Tax Law]
6,client_7,"[Family Law, Labor Law, Civil Rights Law]"
7,client_8,[Estate Planning Law]
8,client_9,"[Criminal Law, Business Law]"
9,client_10,"[Environmental Law, Family Law]"
